In [46]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [48]:
## Get Data

In [49]:
dataset = pd.read_csv('UtilTRNS.csv')

In [50]:
dataset.head()

,Time,sCPU,cCost,sFreq,bCPU,bUtil,bFreq,Mem,mCost,mFreq,GPU,gCost,gFreq
0,1.700423e+09,sCPU,0,1400000,bCPU,0,2000000,Mem,0,825000000,GPU,0,600000000
1,1.700423e+09,sCPU,30,1400000,bCPU,0,2000000,Mem,1,825000000,GPU,0,600000000
2,1.700423e+09,sCPU,32,1400000,bCPU,0,2000000,Mem,0,825000000,GPU,0,600000000
3,1.700423e+09,sCPU,25,1400000,bCPU,0,2000000,Mem,1,825000000,GPU,0,600000000
4,1.700423e+09,sCPU,70,1400000,bCPU,0,2000000,Mem,3,825000000,GPU,0,600000000


In [51]:
new_data = dataset[['cCost','mCost','gCost']]

In [52]:
new_data['shifted_cCost_1'] = new_data['cCost'].shift(1)
new_data['shifted_mCost_1'] = new_data['mCost'].shift(1)
new_data['shifted_gCost_1'] = new_data['gCost'].shift(1)
new_data['shifted_cCost_2'] = new_data['cCost'].shift(2)
new_data['shifted_mCost_2'] = new_data['mCost'].shift(2)
new_data['shifted_gCost_2'] = new_data['gCost'].shift(2)
new_data['Cost'] = dataset['sFreq']/1400000

/tmp/ipykernel_46162/858087496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['shifted_cCost_1'] = new_data['cCost'].shift(1)
/tmp/ipykernel_46162/858087496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['shifted_mCost_1'] = new_data['mCost'].shift(1)
/tmp/ipykernel_46162/858087496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [54]:
new_data = new_data.dropna()
new_data

,cCost,mCost,gCost,shifted_cCost_1,shifted_mCost_1,shifted_gCost_1,shifted_cCost_2,shifted_mCost_2,shifted_gCost_2,Cost
2,32,0,0,30.0,1.0,0.0,0.0,0.0,0.0,1.0
3,25,1,0,32.0,0.0,0.0,30.0,1.0,0.0,1.0
4,70,3,0,25.0,1.0,0.0,32.0,0.0,0.0,1.0
5,100,2,0,70.0,3.0,0.0,25.0,1.0,0.0,1.0
6,100,5,0,100.0,2.0,0.0,70.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
1492,5,3,0,57.0,6.0,0.0,57.0,4.0,0.0,1.0
1493,18,1,0,5.0,3.0,0.0,57.0,6.0,0.0,1.0
1494,1,0,0,18.0,1.0,0.0,5.0,3.0,0.0,1.0
1495,1,0,0,1.0,0.0,0.0,18.0,1.0,0.0,1.0


In [59]:
Y = new_data.iloc[:,0:3]
X = new_data.iloc[:,3:11]
X

,shifted_cCost_1,shifted_mCost_1,shifted_gCost_1,shifted_cCost_2,shifted_mCost_2,shifted_gCost_2,Cost
2,30.0,1.0,0.0,0.0,0.0,0.0,1.0
3,32.0,0.0,0.0,30.0,1.0,0.0,1.0
4,25.0,1.0,0.0,32.0,0.0,0.0,1.0
5,70.0,3.0,0.0,25.0,1.0,0.0,1.0
6,100.0,2.0,0.0,70.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...
1492,57.0,6.0,0.0,57.0,4.0,0.0,1.0
1493,5.0,3.0,0.0,57.0,6.0,0.0,1.0
1494,18.0,1.0,0.0,5.0,3.0,0.0,1.0
1495,1.0,0.0,0.0,18.0,1.0,0.0,1.0


In [58]:
# X = new_data.iloc[1:1496]

In [60]:
X

,shifted_cCost_1,shifted_mCost_1,shifted_gCost_1,shifted_cCost_2,shifted_mCost_2,shifted_gCost_2,Cost
2,30.0,1.0,0.0,0.0,0.0,0.0,1.0
3,32.0,0.0,0.0,30.0,1.0,0.0,1.0
4,25.0,1.0,0.0,32.0,0.0,0.0,1.0
5,70.0,3.0,0.0,25.0,1.0,0.0,1.0
6,100.0,2.0,0.0,70.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...
1492,57.0,6.0,0.0,57.0,4.0,0.0,1.0
1493,5.0,3.0,0.0,57.0,6.0,0.0,1.0
1494,18.0,1.0,0.0,5.0,3.0,0.0,1.0
1495,1.0,0.0,0.0,18.0,1.0,0.0,1.0


In [61]:
Y_val = Y.iloc[:,0:3].values.reshape(-1,3)
Y_val 

array([[32,  0,  0],
       [25,  1,  0],
       [70,  3,  0],
       ...,
       [ 1,  0,  0],
       [ 1,  0,  0],
       [21,  0,  0]])

In [62]:
X_val = X.values.reshape(-1,7)
X_val

array([[30.,  1.,  0., ...,  0.,  0.,  1.],
       [32.,  0.,  0., ...,  1.,  0.,  1.],
       [25.,  1.,  0., ...,  0.,  0.,  1.],
       ...,
       [18.,  1.,  0., ...,  3.,  0.,  1.],
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.]])

In [63]:
X_val.shape

(1495, 7)

In [75]:
regressor = MultiOutputRegressor(SVR(kernel='sigmoid', C=1e3, gamma=0.1)).fit(X_val[:1000], Y_val[:1000])

In [76]:
regressor.fit(X_val[:1000], Y_val[:1000])

MultiOutputRegressor(estimator=SVR(C=1000.0, gamma=0.1, kernel='sigmoid'))

In [77]:
Y_pred = regressor.predict(X_val[1001:1003])

In [81]:
Y_pred

array([[42.1       ,  9.1       , 42.1       ],
       [42.91913433,  9.91913433, 41.28086567]])

In [78]:
regressor.score(X_val[:1000], Y_val[:1000])

-5396.959056356063

In [79]:
regressor.score(X_val[1000:], Y_val[1000:])

-3251.9738942078006

In [82]:
# Y_val[1001:1003]
Y_val[1001:1003]

array([[  1,   8,  68],
       [ 33,  10, 100]])

In [45]:
regressor.RMSE

AttributeError: 'MultiOutputRegressor' object has no attribute 'RMSE'